# 豆瓣用户友邻数据分析
## Part1 数据获取
<font color='red'>*若已有数据集可以跳过此部分，从Part2开始*</font>
### 获取目标用户友邻列表
- 爬取用户首页面友邻页面

  - 从settings中导入目标用户的`uid`
  - 查看豆瓣的友邻页面需要登陆账号，所以使用 cookie 模拟登录

In [1]:
import requests
from bs4 import BeautifulSoup

from settings import user_agent
from settings import cookie
from settings import target_user

session = requests.Session()

url = 'https://www.douban.com/people/' + target_user + '/rev_contacts'
headers = {
    'User-Agent': user_agent,
    'Cookie': cookie,
}

response = session.get(url=url, headers=headers)
if response.status_code != 200:
    print('获取失败，请检查cookie, uid')
soup = BeautifulSoup(response.text, 'lxml')
# print(soup)
print('获取成功')


获取成功


- 从爬取的页面中获取用户的友邻总数

In [2]:
# css 选择器
num = soup.select('#db-usr-profile > div.info > h1')
num = BeautifulSoup(str(num[0]), 'lxml').string
# 数字从倒数第二个字符开始
length = len(num) - 2
while '0' <= num[length] <= '9':
    length -= 1
num = int(num[length + 1:len(num) - 1])
print(num)


459


- 爬取所有友邻列表页面，获取所有友邻的`uid`，保存至本地

In [3]:
from settings import uid_file

with open(uid_file, 'w') as file:
    # 每页显示最多 70 个友邻
    for i in range(0, num, 70):
        current_url = url + '?start=' + str(i)
        # print(current_url)
        response = session.get(url=current_url, headers=headers)
        soup = BeautifulSoup(response.text, 'lxml')
        peoples = soup.select('#content > div > div.article > dl > dt > a')
        for people in peoples:
            uid = (BeautifulSoup(str(people), 'lxml').a['href'])[30:-1]
            print(uid)
            file.write(uid + '\n')


dengliang
gustwind
178563230
67795186
cctan123
xiaorun95
95312530
kex
guohaoyu110
73643080
1604979
34522588
moran8008
55998154
ilicis
37794565
31292509
120691876
165305791
73491411
53485231
172230839
tongtong_Ivu
46499275
190604849
67439343
lzonline01
190504329
jgs
2887413
173493508
160090199
167202000
163669931
151739065
179257911
63630107
zhuoxiaomo
46280525
87493724
47478408
158373086
typhanie
54523318
68635733
amyli1989
DAIZI90
1017708
55938120
47091718
33327166
48911840
52941590
giazen
180125257
62518964
gunillazhang
ofRaphael
echodreaming
suixh
65769663
24044869
80239729
55745235
80588107
littleqing
102805338
th507
53966201
129683084
180198556
65673386
twilightforest
75133029
52067931
157507056
donggehaha
28260229
Homer.J
167631346
lichcas
53802947
191482190
191035229
160114771
169208427
54458544
190797995
sevencakes
149183349
diqiuren315
96374578
bestjyz
53060443
tyfulcrum
49397273
181298480
77572301
lilpump
ahachi
189849663
48481225
174353285
40357173
moniterluo
yetone
ma233
38

### 获取友邻用户数据
- 通过[https://m.douban.com/people/uid/subject_profile](https://m.douban.com/people/uid/subject_profile)获取账户信息
- 通过[https://m.douban.com/people/uid/movie_charts](https://m.douban.com/people/uid/movie_charts)获取观影信息

In [4]:
from json import loads


def get_movie_info(nuid=''):
    nurl = 'https://m.douban.com/rexxar/api/v2/user/' + nuid + '/collection_stats?type=movie&for_mobile=1&ck=5Kvd'
    nreferer = 'https://m.douban.com/people/' + nuid + '/movie_charts'
    nheaders = {
        'Referer': nreferer,
        'User-Agent': user_agent,
    }
    nresponse = session.get(url=nurl, headers=nheaders)
    # 返回的数据为 json 格式，使用 loads 解析
    ndecoded = loads(nresponse.text)
    # print(decoded)
    nrow = []
    # 观影数
    try:
        nrow.append(ndecoded['total_collections'])
    except:
        nrow.append('')
    # 观看时间
    try:
        nrow.append(int(ndecoded['total_spent']))
    except:
        nrow.append('')
    # 消费
    try:
        nrow.append(int(ndecoded['total_cost']))
    except:
        nrow.append('')
    # 平均每周观看时间
    try:
        nrow.append(round(ndecoded['weekly_avg'], 1))
    except:
        nrow.append('')
    # 以下两项为最常观看地区
    try:
        nrow.append(ndecoded['countries'][0]['name'])
    except:
        nrow.append('')
    try:
        nrow.append(ndecoded['countries'][1]['name'])
    except:
        nrow.append('')
    # 以下三项为最常观看类型
    try:
        nrow.append(ndecoded['genres'][0]['name'])
    except:
        nrow.append('')
    try:
        nrow.append(ndecoded['genres'][1]['name'])
    except:
        nrow.append('')
    try:
        nrow.append(ndecoded['genres'][2]['name'])
    except:
        nrow.append('')
    # print(row)
    return nrow


def get_user_info(nuid=''):
    nurl = 'https://m.douban.com/rexxar/api/v2/user/' + nuid + '/archives_summary?for_mobile=1&ck=5Kvd'
    nreferer = 'https://m.douban.com/people/' + nuid + '/subject_profile'
    nheaders = {
        'Referer': nreferer,
        'User-Agent': user_agent,
    }
    nresponse = session.get(url=nurl, headers=nheaders)
    # 返回的数据为 json 格式，使用 loads 解析
    ndecoded = loads(nresponse.text)
    # print(decoded)
    nrow = []
    # 用户所在地区
    try:
        nrow.append(ndecoded['user']['loc']['name'])
    except:
        nrow.append('')
    # 用户广播数
    try:
        nrow.append(ndecoded['user']['statuses_count'])
    except:
        nrow.append('')
    # 用户注册时间
    try:
        nrow.append(ndecoded['user']['reg_time'][:4])
    except:
        nrow.append('')
    # 用户性别
    try:
        nrow.append(ndecoded['user']['gender'])
    except:
        nrow.append('')
    # print(row)
    return nrow


def get_info(nuid=''):
    nrow = []
    nrow += get_user_info(nuid)
    nrow += get_movie_info(nuid)
    print(nrow)
    return nrow


- 读取本地`uid`文件，获取所有友邻数据，写入数据集

In [5]:
import csv
from time import sleep

from settings import csv_title
from settings import dataset_file

with open(uid_file) as infile:
    with open(dataset_file, 'w', encoding='utf-8', newline='') as outfile:
        csv_file = csv.writer(outfile, dialect='excel')
        csv_file.writerow(csv_title)
        for line in infile:
            uid = line[:-1]
            # print(uid)
            csv_file.writerow(get_info(uid))
            sleep(2)


['扬州', 983, '2007', '', 25, 57, 910, 0.0, '中国大陆', '美国', '剧情', '科幻', '动作']
['南京', 903, '2010', '', 6, '', '', '', '', '', '', '', '']
['Tbilisi', 168, '2018', 'M', 49, 250, 140, 0.8, '日本', '美国', '动画', '剧情', '奇幻']
['盘锦', 67, '2013', 'M', 75, 232, 3990, 0.2, '美国', '中国大陆', '动作', '剧情', '冒险']
['上海', 697, '2010', '', 431, 955, 2870, 1.0, '美国', '英国', '剧情', '喜剧', '惊悚']
['上海', 415, '2014', 'M', 223, 977, 5880, 0.9, '美国', '中国大陆', '剧情', '动作', '冒险']
['Milano', 558, '2014', 'M', 19, 60, 280, 0.1, '美国', '日本', '剧情', '传记', '悬疑']
['深圳', 1555, '2010', '', 933, 2496, 7070, 2.0, '美国', '日本', '剧情', '动作', '惊悚']
['扬州', 1416, '2018', '', 377, 992, 5250, 5.1, '美国', '英国', '剧情', '动作', '犯罪']
['武汉', 164, '2013', '', 7, '', '', '', '', '', '', '', '']
['北京', 305, '2007', '', 236, 1265, 1050, 0.4, '美国', '日本', '剧情', '爱情', '喜剧']
['深圳', 341, '2010', 'M', 85, 240, 2380, 0.2, '美国', '中国大陆', '剧情', '奇幻', '喜剧']
['上海', 429, '2008', 'M', 304, 1460, 8190, 0.5, '美国', '中国大陆', '剧情', '喜剧', '动作']
['哈尔滨', 1094, '2011', 'F', 333, 1580, 

['上海', 1054, '2007', '', 33, 243, 420, 0.1, '美国', '日本', '剧情', '喜剧', '爱情']
['香港', 394, '2018', 'F', 965, 7552, 10010, 20.1, '中国大陆', '香港', '剧情', '动作', '喜剧']
['上海', 437, '2008', 'F', 3, '', '', '', '', '', '', '', '']
['北京', 1669, '2016', 'F', 128, 289, 4830, 0.8, '美国', '中国大陆', '剧情', '动作', '冒险']
['北京', 6580, '2011', 'F', 1099, 7063, 11270, 2.7, '美国', '日本', '剧情', '喜剧', '爱情']
['广州', 676, '2017', 'M', 197, 487, 1680, 1.6, '美国', '英国', '剧情', '惊悚', '喜剧']
['上海', 5123, '2010', '', 415, 964, 5880, 0.9, '美国', '日本', '剧情', '爱情', '喜剧']
['', 1157, '2017', '', 405, 2007, 4550, 4.8, '日本', '中国大陆', '剧情', '动画', '喜剧']
['成都', 357, '2014', 'F', 63, 195, 910, 0.3, '美国', '中国大陆', '剧情', '爱情', '同性']
['上海', 173, '2013', 'U', 46, 164, 560, 0.2, '日本', '美国', '剧情', '爱情', '喜剧']
['忻州', 1734, '2015', 'M', 232, 984, 560, 1.2, '美国', '日本', '剧情', '惊悚', '科幻']
['福州', 772, '2014', 'M', 46, 171, 1050, 0.2, '日本', '中国大陆', '剧情', '动画', '科幻']
['', 158, '2018', '', 382, 3697, 1470, 11.6, '日本', '香港', '剧情', '喜剧', '爱情']
['泰安', 978, '2011',

['无锡', 55, '2011', '', 306, 866, 5040, 0.8, '美国', '中国大陆', '剧情', '爱情', '喜剧']
['上海', 3515, '2007', '', 1724, 4615, 14770, 2.8, '美国', '英国', '剧情', '动作', '惊悚']
['', '', '', '', '', '', '', '', '', '', '', '', '']
['北京', 78, '2015', '', 107, 301, 560, 0.6, '美国', '英国', '剧情', '爱情', '喜剧']
['广州', 2138, '2007', 'F', 182, 1372, 3290, 0.3, '美国', '日本', '剧情', '动画', '爱情']
['广州', 652, '2009', 'U', 1654, 5654, 20860, 3.1, '美国', '中国大陆', '剧情', '喜剧', '爱情']
['', '', '', '', '', '', '', '', '', '', '', '', '']
['成都', 111, '2014', '', 7, '', '', '', '', '', '', '', '']
['Waterloo', 1, '2010', 'M', 733, 3866, 8540, 1.6, '美国', '中国大陆', '剧情', '喜剧', '爱情']
['南京', 2668, '2017', 'U', 865, 3017, 17640, 7.3, '美国', '中国大陆', '剧情', '喜剧', '动画']
['', '', '', '', '', '', '', '', '', '', '', '', '']
['大连', 916, '2010', 'F', 581, 2682, 5250, 1.3, '日本', '中国大陆', '剧情', '爱情', '喜剧']
['杭州', 2451, '2008', 'M', 343, 1161, 7350, 0.6, '美国', '中国大陆', '剧情', '动作', '喜剧']
['厦门', 527, '2012', '', 66, 629, 140, 0.2, '韩国', '美国', '剧情', '爱情', '喜剧']

['上海', 11795, '2008', 'M', 765, 3627, 8260, 1.3, '美国', '日本', '剧情', '喜剧', '动作']
['上海', 12355, '2011', '', 2941, 12570, 18270, 7.1, '美国', '日本', '剧情', '动画', '喜剧']
['San Francisco Bay Area', 2958, '2008', 'M', 457, 1381, 4900, 0.8, '美国', '中国大陆', '剧情', '爱情', '喜剧']
['北京', 394, '2016', '', 62, 297, 1470, 0.5, '美国', '中国大陆', '剧情', '冒险', '奇幻']
['北京', 620, '2015', 'F', 465, 1380, 14000, 2.0, '美国', '中国大陆', '剧情', '动作', '喜剧']
['石家庄', 405, '2016', 'U', 147, 369, 2380, 1.1, '美国', '日本', '剧情', '喜剧', '犯罪']
['杭州', 3474, '2009', 'F', 2899, 15983, 18340, 5.6, '日本', '中国大陆', '剧情', '爱情', '喜剧']
['重庆', 869, '2012', '', 84, 565, 350, 0.2, '美国', '日本', '动画', '喜剧', '剧情']
['北京', 1016, '2010', 'M', 794, 3470, 10500, 1.7, '美国', '中国大陆', '动作', '剧情', '喜剧']
['郑州', 2251, '2010', 'M', 217, 612, 910, 0.6, '美国', '中国大陆', '剧情', '喜剧', '爱情']
['成都', 1744, '2008', 'M', 896, 2209, 11550, 1.5, '美国', '中国大陆', '剧情', '喜剧', '爱情']
['杭州', 596, '2011', 'U', 60, 238, 630, 0.1, '日本', '美国', '剧情', '爱情', '喜剧']
['杭州', 1373, '2011', '', 197, 542, 24

['上海', 126, '2016', '', 20, 43, 770, 0.2, '美国', '澳大利亚', '剧情', '喜剧', '动作']
['株洲', 5194, '2008', 'U', 1089, 3235, 15890, 2.0, '中国大陆', '美国', '剧情', '爱情', '喜剧']
['丽水', 5122, '2008', 'F', 1099, 6103, 10080, 1.8, '美国', '日本', '剧情', '动画', '喜剧']
['', '', '', '', '', '', '', '', '', '', '', '', '']
['Kosovo', 1806, '2015', 'U', 22, 49, 280, 0.1, '美国', '伊朗', '剧情', '家庭', '悬疑']
['上海', 2597, '2009', '', 491, 1973, 7280, 1.0, '美国', '日本', '剧情', '动作', '冒险']
['台北', 229, '2012', 'F', 1245, 3004, 7700, 3.4, '美国', '英国', '剧情', '喜剧', '惊悚']
['上海', 12450, '2010', 'M', 1201, 6057, 8680, 2.7, '美国', '日本', '剧情', '动画', '喜剧']
['上海', 4064, '2013', 'M', 57, 193, 770, 0.2, '日本', '美国', '剧情', '喜剧', '爱情']
['Los Angeles', 740, '2013', '', 186, 453, 3290, 0.6, '美国', '日本', '剧情', '冒险', '科幻']
['上海', 5863, '2008', '', 39, 96, 910, 0.1, '日本', '美国', '剧情', '动画', '科幻']
['香港', 2833, '2009', 'M', 864, 2485, 11060, 1.7, '美国', '日本', '剧情', '喜剧', '动作']
['New York City', 2220, '2016', 'F', 369, 1740, 3710, 2.5, '美国', '日本', '剧情', '喜剧', '爱情'